In [1]:
from time import sleep
from tdmclient import ClientAsync, aw
client = ClientAsync()


In [2]:
client.process_waiting_messages()

True

In [3]:
node = client.nodes[0]

In [4]:
aw(node.lock())

Node 87d48be9-2b09-4962-aab4-b8705c0cc9e7

In [5]:
aw(node.unlock())

In [6]:
aw(node.lock())

Node 87d48be9-2b09-4962-aab4-b8705c0cc9e7

In [7]:
async def set_motors(speed_left, speed_right):
    speeds = {
        "motor.left.target": [speed_left],
        "motor.right.target": [speed_right],
    }
    await node.set_variables(speeds)

async def get_prox():
    await node.wait_for_variables({"prox.horizontal"})
    await client.sleep(0.05)
    return list(node.v.prox.horizontal)



In [11]:
await set_motors(50,50)

In [12]:
await set_motors(0,0)

In [1]:
import time

PROX_FARLEFT = 0
PROX_LEFT = 1
PROX_FRONT = 2
PROX_RIGHT = 3
PROX_FARRIGHT = 4

DETECTION_THRESHOLD = 2000
T_TURN = 0.5
T_DRIVE = 4.5
TURN_RATION = 5

avoiding_left = False
avoiding_right = False
avoiding_stage = 0
speed = 100
start_time = 0
current_time = 0

while True:
    prox_values = await get_prox()
    prox_farleft = prox_values[PROX_FARLEFT]
    prox_left = prox_values[PROX_LEFT]
    prox_front = prox_values[PROX_FRONT]
    prox_right = prox_values[PROX_RIGHT]
    prox_farright = prox_values[PROX_FARRIGHT]

    current_time = time.time()
    
    if avoiding_left or avoiding_right:
        if avoiding_left:
            if avoiding_stage == 1:
                await set_motors(speed, -speed)
                if current_time - start_time > T_TURN:
                    avoiding_stage = 2
                    start_time = time.time()
                    await set_motors(0, 0)
            if avoiding_stage == 2:
                await set_motors(speed, speed)
                if current_time - start_time > T_DRIVE: 
                    avoiding_stage = 3
                    start_time = time.time()
                    await set_motors(0, 0)

            if avoiding_stage == 3:
                await set_motors(-speed, speed)
                if current_time - start_time > TURN_RATION*T_TURN:
                    avoiding_stage = 4
                    start_time = time.time()
                    await set_motors(0,0)

            if avoiding_stage == 4:
                await set_motors(speed, speed)
                if current_time - start_time > T_DRIVE:
                    avoiding_stage = 0
                    avoiding_left = False
                    start_time = 0
                    await set_motors(0,0)
        else:
            if avoiding_stage == 1:
                await set_motors(-speed, speed)
                if current_time - start_time > T_TURN:
                    avoiding_stage = 2
                    start_time = time.time()
                    await set_motors(0, 0)
            if avoiding_stage == 2:
                await set_motors(speed, speed)
                if current_time - start_time > T_DRIVE: 
                    avoiding_stage = 3
                    start_time = time.time()
                    await set_motors(0, 0)

            if avoiding_stage == 3:
                await set_motors(speed, -speed)
                if current_time - start_time > TURN_RATION*T_TURN:
                    avoiding_stage = 4
                    start_time = time.time()
                    await set_motors(0,0)

            if avoiding_stage == 4:
                await set_motors(speed, speed)
                if current_time - start_time > T_DRIVE:
                    avoiding_stage = 0
                    avoiding_right = False
                    start_time = 0
                    await set_motors(0,0)
            
    
    if prox_farleft + prox_left + prox_front + prox_right + prox_farright > DETECTION_THRESHOLD:
        if prox_farleft + prox_left + prox_front >= prox_farright + prox_right + prox_front:
            avoiding_left = True
            avoiding_stage = 1
            start_time = time.time()
        else:
            avoiding_right = True
            avoiding_stage = 1
            start_time = time.time()
    elif !avoiding_left and !avoiding_right:
        await set_motors(0,0)
            
    

SyntaxError: invalid syntax (1822387724.py, line 99)

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
